In [1]:
import random
import anvil.server

In [1]:
class Validate():
    def __init__(self,feed=None,verbose=True):
        self.dfD={}
        self.verbose=verbose
        self.feed=feed
        if feed!=None:
            n=random.randint(3,10)
            t=random.randint(5,10)
            feed.init_counters()
            for j in range(t): feed.step()
            di=random.randint(0,len(feed.dates)-1)
            for i in range(n):
                ti=random.randint(0,len(feed.tickers)-1)
                self.dfD[feed.tickers[ti]]=feed.getDataN(feed.tickers[ti],feed.dates[di])
    def remote_check_entry_batch(dfD,rpcname):
        if self.feed==None: return [{}]
        # c = rpyc.connect("localhost", 18861)
        dfR={}
        for t in dfD.keys():
            dfR[t]=dfD[t].to_dict('records')
        return anvil.server.call('check_entry_batch_'+rpcname,dfR)
        # return c.root.check_entry(pickle.dumps(dfR),rpcname)       
    def validate(self,rpcname=None,retval=None,dfD=None):
        if dfD==None: dfD=self.dfD
        if rpcname!=None: retval=Validate.remote_check_entry_batch(dfD,rpcname)
        if type(retval)!=list: return('must return list')
        elif len(retval)!=3: return('must return list of three dictionaries')
        elif False in [type(le)==dict for le in retval]:
            msg='must return list of dictionaries'
            if self.verbose: print(msg)
            return -1,msg
        elif False in [k in dfD for le in retval for k in le]:
            msg='dictionary keys must be in input'
            if self.verbose:print(msg)
            return -1,msg
        elif False in [type(le[k])==int for le in retval for k in le]:
            msg='values must be integers'
            if self.verbose: print(msg)
            return -1,msg
        elif False in [k in retval[i] for k in dfD for i in range(3)]:
            msg='quant,stop and target must be returned for each ticker in dfD'
            if self.verbose: print(msg) 
            return -1,msg
        else: 
            msg='valid return from remote call'
            if self.verbose: print(msg)
            return 0,msg